In [1]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import os
import pandas as pd
from pathlib import Path
from sqlutils import sqlutils
import numpy as np

In [2]:
# Déterminer le chemin du script
script_path = Path().resolve()

# # Select des avis depuis la base de données et depuis le chemin du script
bdd = sqlutils(script_path / "../../data/friands.db")
success, t_avis = bdd.select("SELECT * FROM avis")

if not success:
    print("Erreur lors de l'extraction des avis depuis la base de données")
    print(t_avis)

# # Insérer les champs extraits de la base de données dans un dataframe
df = pd.DataFrame(
    t_avis,
    columns=[
        "id_avis",
        "id_restaurant",
        "nom_utilisateur",
        "note_restaurant",
        "date_avis",
        "titre_avis",
        "contenu_avis",
        "label",
    ],
)

In [3]:
# Chargement du modèle français de spaCy
nlp = spacy.load("fr_core_news_md")

In [ ]:
def preprocess_reviews(reviews):
    # Nettoyage et préparation des textes
    processed_reviews = []
    for review in reviews:
        doc = nlp(review.lower())
        # Garder uniquement les tokens pertinents (noms, adjectifs, verbes)
        tokens = [
            token.lemma_
            for token in doc
            if not token.is_stop
            and not token.is_punct
            and token.pos_ in ["NOUN", "ADJ", "VERB"]
        ]
        processed_reviews.append(" ".join(tokens))
    return processed_reviews


def analyze_restaurants(df):
    # Prétraitement des avis
    processed_reviews = (
        df.groupby("restaurant_id")["review"]
        .apply(lambda x: " ".join(preprocess_reviews(x)))
        .reset_index()
    )

    # Vectorisation TF-IDF
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(processed_reviews["review"])

    # Clustering des restaurants
    n_clusters = 5  # À ajuster selon vos besoins
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(tfidf_matrix)

    # Analyse des caractéristiques principales par cluster
    feature_names = vectorizer.get_feature_names_out()
    cluster_keywords = []
    for i in range(n_clusters):
        center = kmeans.cluster_centers_[i]
        top_keywords = [feature_names[j] for j in center.argsort()[-10:][::-1]]
        cluster_keywords.append(top_keywords)

    # Calcul de similarité entre restaurants
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return clusters, similarity_matrix, cluster_keywords


def analyze_sentiment(reviews):
    # Analyse de sentiment avec NLTK
    sia = SentimentIntensityAnalyzer()
    sentiments = [sia.polarity_scores(review)["compound"] for review in reviews]
    return np.mean(sentiments)

In [ ]:
# Supposons que nous avons un DataFrame avec les colonnes: restaurant_id, review
restaurants_df = df[["id_restaurant", "contenu_avis"]].rename(
    columns={"id_restaurant": "restaurant_id", "contenu_avis": "review"}
)

# Analyse globale
clusters, similarity_matrix, cluster_keywords = analyze_restaurants(restaurants_df)

# Ajout des résultats au DataFrame
restaurants_df["cluster"] = clusters
restaurants_df["sentiment_score"] = restaurants_df.groupby("restaurant_id")[
    "review"
].apply(analyze_sentiment)

# Classification des restaurants
results = pd.DataFrame(
    {
        "restaurant_id": restaurants_df["restaurant_id"].unique(),
        "cluster": clusters,
        "avg_sentiment": restaurants_df.groupby("restaurant_id")[
            "sentiment_score"
        ].mean(),
        "cluster_keywords": [cluster_keywords[c] for c in clusters],
    }
)

print(results)

ValueError: Length of values (10) does not match length of index (4573)

In [9]:
print(clusters)

[1 2 0 0 1 3 1 0 4 1]


In [10]:
# Créer une nouvelle colonne cluster dans le dataframe restaurants_df, et y injecter les valeurs de clusters
restaurants_df["cluster"] = clusters

ValueError: Length of values (10) does not match length of index (4573)

In [11]:
import pandas as pd

# Créons d'abord un DataFrame d'exemple
dfa = pd.DataFrame({"A": [1, 2, 3], "B": ["a", "b", "c"]})

# Méthode 1 : Assignation directe
nouvelle_liste = [10, 20, 30]
dfa["Nouvelle_Colonne"] = nouvelle_liste

# Méthode 2 : Utiliser insert() pour spécifier la position
nouvelle_liste2 = [100, 200, 300]
dfa.insert(loc=1, column="Autre_Colonne", value=nouvelle_liste2)

In [12]:
print(dfa)

   A  Autre_Colonne  B  Nouvelle_Colonne
0  1            100  a                10
1  2            200  b                20
2  3            300  c                30


In [14]:
print(restaurants_df)

      restaurant_id                                             review
0                 1  Agréable petit restaurant à la déco typique,la...
1                 1  Ce restaurant est une magnifique découverte. L...
2                 1  Lendroit ne paie pas de mine mais les plats so...
3                 1  Un très bon moment, les plats sont très bons e...
4                 1  1ère et dernière fois  Jai commandé un Donbury...
...             ...                                                ...
4568             10  Le menu est très riche avec des bons plats, bo...
4569             10  Que dire, un service impeccable, un décor épur...
4570             10  très bon accueil plat excellent, décor moderne...
4571             10  Un nouveau petit restaurant intimiste, géré pa...
4572             10  Nous étions à Lyon dans le cadre dun mois. Nou...

[4573 rows x 2 columns]
